In [1]:
!pip install sentence_transformers datasets sentencepiece

  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/ba/20/7ef81df2e07322d95332d07c1c38c597f543c1f666d689a3153ba6fa09e3/sentence_transformers-2.6.1-py3-none-any.whl.metadata
  Using cached sentence_transformers-2.6.1-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/95/fc/661a7f06e8b7d48fcbd3f55423b7ff1ac3ce59526f146fda87a1e1788ee4/datasets-2.18.0-py3-none-any.whl.metadata
  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/fb/12/2f5c8d4764b00033cf1c935b702d3bb878d10be9f0b87f0253495832d85f/sentencepiece-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached sentencepiece-0.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Obtaining dependency information for transformers<5.0.0,>=4.32

In [2]:
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
import math
from datasets import load_dataset

In [3]:
import sentencepiece
import pandas as pd

## Load the Dataset

In [4]:
# Load and prepare the SNLI dataset
dataset = load_dataset("paws", "labeled_final")
dataset = dataset.filter(lambda x: x['label'] != -1)

In [5]:
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])

In [6]:
dataset["train"][0]

{'id': 1,
 'sentence1': 'In Paris , in October 1560 , he secretly met the English ambassador , Nicolas Throckmorton , asking him for a passport to return to England through Scotland .',
 'sentence2': 'In October 1560 , he secretly met with the English ambassador , Nicolas Throckmorton , in Paris , and asked him for a passport to return to Scotland through England .',
 'label': 0}

In [7]:
# Print first row from train set
print(df_train.iloc[0]) 

id                                                           1
sentence1    In Paris , in October 1560 , he secretly met t...
sentence2    In October 1560 , he secretly met with the Eng...
label                                                        0
Name: 0, dtype: object


## Convert dataset to list of InputExamples


In [8]:
unique_train_labels = df_train['label'].unique()
unique_test_labels = df_test['label'].unique()

print("Unique labels in training data:", unique_train_labels)
print("Unique labels in testing data:", unique_test_labels)

Unique labels in training data: [0 1]
Unique labels in testing data: [0 1]


In [9]:
train_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=int(row['label'])) for index, row in df_train.iterrows()]
test_examples = [InputExample(texts=[row['sentence1'], row['sentence2']], label=int(row['label'])) for index, row in df_test.iterrows()]


In [10]:
try:
    test_evaluator = CEBinaryClassificationEvaluator.from_input_examples(test_examples, name='test_eval')
except AssertionError as e:
    print("Error with labels:", e)

In [11]:
print(train_examples[0])

<InputExample> label: 0, texts: In Paris , in October 1560 , he secretly met the English ambassador , Nicolas Throckmorton , asking him for a passport to return to England through Scotland .; In October 1560 , he secretly met with the English ambassador , Nicolas Throckmorton , in Paris , and asked him for a passport to return to Scotland through England .


In [12]:
test_evaluator = CEBinaryClassificationEvaluator.from_input_examples(test_examples, name='test_eval')


In [13]:
len(train_examples)


49401

In [14]:
! nvidia-smi

Tue Apr 16 04:43:51 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:87:00.0 Off |                  Off |
| 30%   31C    P8             28W /  300W |       0MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Model Setup

In [15]:
num_epochs = 10
model_save_path = "./model_dump"
model_name = 'cross-encoder/nli-deberta-v3-base' # base model, use 'vectara/hallucination_evaluation_model' if you want to further fine-tune ours

model = CrossEncoder(model_name, num_labels=1, automodel_args={'ignore_mismatched_sizes':True})



/opt/conda/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-deberta-v3-base and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/transformers/convert_slow_to

## Training the model


In [16]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=128)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
model.fit(train_dataloader=train_dataloader,
          evaluator=test_evaluator,
          epochs=num_epochs,
          evaluation_steps=10_000,
          warmup_steps=warmup_steps,
          output_path=model_save_path,
          show_progress_bar=True)

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

Iteration:   0%|          | 0/386 [00:00<?, ?it/s]

## Train the model


In [18]:
model_save_path = "./model_output"


In [19]:
model.save(model_save_path)


## Load the model

In [20]:
from sentence_transformers import CrossEncoder


# Load the model from the path
model = CrossEncoder(model_save_path)


In [21]:
def predict_with_model(model, sentence1, sentence2):
    # Directly use the predict method without setting eval
    with torch.no_grad():  # It's still good practice to use no_grad() to disable gradient calculation
        predictions = model.predict([(sentence1, sentence2)])
    return predictions


In [32]:
import torch
# Assuming the model is already loaded and available as `model`
sentence1 = "The cat sits inside the house."
sentence2 = "A cat is sitting in the kitchen."
prediction = predict_with_model(model, sentence1, sentence2)
print("Prediction:", prediction)


Prediction: [0.61624146]


In [23]:
# Example 2

# Assuming the model is already loaded and available as `model`
sentence1 = "The profit generated in 2022 is 26 million dollars"
sentence2 = "A profit generated in 2022 is $25.00 million"
prediction = predict_with_model(model, sentence1, sentence2)
print("Prediction:", prediction)


Prediction: [0.00872917]


In [47]:

# Load the model from the saved directory
model = CrossEncoder(model_save_path)

In [24]:
def predict_with_model(model, sentence1, sentence2):
    # Use the model's predict method which expects a list of sentence pairs
    predictions = model.predict([(sentence1, sentence2)])
    return predictions

# Example usage
sentence1 = "Einstein is from Germany."
sentence2 = "Einstein is from Berlin, Gemany."
prediction = predict_with_model(model, sentence1, sentence2)
print("Prediction Score:", prediction)


Prediction Score: [0.98758197]


## Testing the model 

In [29]:
import torch

def calculate_accuracy(model, dataloader):
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Deactivate gradients for the following code
        for texts, labels in dataloader:
            predictions = model.predict(texts)  # This assumes that 'predict' returns a list or array of probabilities
            
            # Convert predictions to a tensor if not already one and apply threshold
            predictions_tensor = torch.tensor(predictions)
            predicted_labels = (predictions_tensor >= 0.5).long()  # Classify predictions based on the 0.5 threshold
            #import pdb;pdb.set_trace()
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)

    accuracy = correct_predictions / total_predictions
    return accuracy


In [30]:
from torch.utils.data import DataLoader


def custom_collate_fn(batch):
    texts = [[ex.texts[0], ex.texts[1]] for ex in batch]  # Extract texts from each InputExample
    labels = torch.tensor([ex.label for ex in batch])  # Extract labels and convert to tensor
    return texts, labels


test_dataloader = DataLoader(test_examples, batch_size=128, collate_fn=custom_collate_fn)




In [31]:
test_accuracy = calculate_accuracy(model, test_dataloader)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.94875


In [34]:
test_examples[0].texts

['This was a series of nested angular standards , so that measurements in azimuth and elevation could be done directly in polar coordinates relative to the ecliptic .',
 'This was a series of nested polar scales , so that measurements in azimuth and elevation could be performed directly in angular coordinates relative to the ecliptic .']

In [36]:
test_examples[0].label

0

In [34]:
!pip install torchmetrics

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for torchmetrics from https://files.pythonhosted.org/packages/f3/0e/cedcb9c8aeb2d1f655f8d05f841b14d84b0a68d9f31afae4af55c7c6d0a9/torchmetrics-1.3.2-py3-none-any.whl.metadata
  Using cached torchmetrics-1.3.2-py3-none-any.whl.metadata (19 kB)
  Obtaining dependency information for lightning-utilities>=0.8.0 from https://files.pythonhosted.org/packages/5e/9e/e7768a8e363fc6f0c978bb7a0aa7641f10d80be60000e788ef2f01d34a7c/lightning_utilities-0.11.2-py3-none-any.whl.metadata
  Using cached lightning_utilities-0.11.2-py3-none-any.whl.metadata (4.7 kB)
Using cached torchmetrics-1.3.2-py3-none-any.whl (841 kB)
Using cached lightning_utilities-0.11.2-py3-none-any.whl (26 kB)


In [39]:
import torch

def calculate_metrics(model, dataloader):
    correct_predictions = 0
    total_predictions = 0
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    with torch.no_grad():  # Deactivate gradients for the following code
        for texts, labels in dataloader:
            predictions = model.predict(texts)  # Assumes 'predict' returns a list or array of probabilities
            
            predictions_tensor = torch.tensor(predictions)
            predicted_labels = (predictions_tensor >= 0.5).long()  # Classify predictions based on the 0.5 threshold
            
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)

            # Calculate true positives, false positives, and false negatives
            true_positives += ((predicted_labels == 1) & (labels == 1)).sum().item()
            false_positives += ((predicted_labels == 1) & (labels == 0)).sum().item()
            false_negatives += ((predicted_labels == 0) & (labels == 1)).sum().item()

    # Calculate accuracy, precision, recall, and F1 score
    accuracy = correct_predictions / total_predictions
    precision = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return accuracy, precision, recall, f1

# Example usage assuming the dataloader and model are set up:
accuracy, precision, recall, f1 = calculate_metrics(model, test_dataloader)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Accuracy: 0.94875
Precision: 0.9282191780821918
Recall: 0.9581447963800905
F1 Score: 0.9429446145282494


## Uploading it on Huggingface

In [40]:
!pip install huggingface_hub


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!huggingface-cli login


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
from huggingface_hub import Repository, HfFolder

# Define paths and repository details
local_dir = 'training'  # Local directory where your model and README.md are saved
repo_name = 'hallucination_detector'  # Repository name on Hugging Face Hub
username = 'Varun-Sayapaneni'  # Your Hugging Face username

# Create or use existing repository
repo = Repository(local_dir=local_dir, clone_from=f'{username}/{repo_name}', use_auth_token=True)

# If you need to add files, you can do it here
# For example:
# shutil.copy('my_model.bin', local_dir)

# Commit and push files to the repository
repo.git_add('*')
repo.git_commit("Initial model upload")
repo.git_push()